# Importing libraries

In [176]:
import pandas as pd 
import numpy as np
import pymysql
from sqlalchemy import create_engine


df = pd.read_csv('business_loan_data.csv')

## Analysis of the Business Loan Dataset

In [177]:
df.columns

Index(['id', 'address_state', 'application_type', 'emp_length', 'emp_title',
       'grade', 'home_ownership', 'issue_date', 'last_credit_pull_date',
       'last_payment_date', 'loan_status', 'next_payment_date', 'member_id',
       'purpose', 'sub_grade', 'term', 'verification_status', 'annual_income',
       'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc',
       'total_payment'],
      dtype='object')

In [178]:
df.shape

(40000, 24)

In [179]:
df.head()

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1000000,West Bengal,INDIVIDUAL,4 years,Larsen & Toubro,E,OTHER,9/26/2024,7/29/2024,6/22/2024,...,D3,60 months,Verified,87905,0.0691,589.27,0.1917,41161,7,11704
1,1000001,Telangana,INDIVIDUAL,10 years,Britannia Industries,C,MORTGAGE,4/26/2024,06/08/2024,4/20/2024,...,F2,60 months,Source Verified,35768,0.0683,214.44,0.1225,45430,3,16223
2,1000002,Maharashtra,INDIVIDUAL,< 1 year,Bharti Airtel,B,OTHER,3/23/2024,1/25/2025,01/09/2024,...,C5,60 months,Source Verified,95008,0.0612,918.99,0.1847,28540,14,12449
3,1000003,Madhya Pradesh,INDIVIDUAL,3 years,Dabur India,C,MORTGAGE,8/22/2024,04/05/2024,12/14/2024,...,D4,36 months,Verified,54660,0.1953,617.99,0.1583,6469,7,16872
4,1000004,Tamil Nadu,INDIVIDUAL,4 years,Wipro,E,RENT,03/02/2024,6/25/2024,7/31/2024,...,A1,60 months,Not Verified,103601,0.2884,142.59,0.1540,33992,20,11348


In [180]:
df.isnull().sum()

id                       0
address_state            0
application_type         0
emp_length               0
emp_title                0
grade                    0
home_ownership           0
issue_date               0
last_credit_pull_date    0
last_payment_date        0
loan_status              0
next_payment_date        0
member_id                0
purpose                  0
sub_grade                0
term                     0
verification_status      0
annual_income            0
dti                      0
installment              0
int_rate                 0
loan_amount              0
total_acc                0
total_payment            0
dtype: int64

In [181]:
df.duplicated().sum()

np.int64(0)

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     40000 non-null  int64  
 1   address_state          40000 non-null  object 
 2   application_type       40000 non-null  object 
 3   emp_length             40000 non-null  object 
 4   emp_title              40000 non-null  object 
 5   grade                  40000 non-null  object 
 6   home_ownership         40000 non-null  object 
 7   issue_date             40000 non-null  object 
 8   last_credit_pull_date  40000 non-null  object 
 9   last_payment_date      40000 non-null  object 
 10  loan_status            40000 non-null  object 
 11  next_payment_date      40000 non-null  object 
 12  member_id              40000 non-null  int64  
 13  purpose                40000 non-null  object 
 14  sub_grade              40000 non-null  object 
 15  te

## Data Cleaning and Standardization

### fixing alingment of data to left

In [183]:
pd.options.display.colheader_justify = 'left'

df.head()

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1000000,West Bengal,INDIVIDUAL,4 years,Larsen & Toubro,E,OTHER,9/26/2024,7/29/2024,6/22/2024,...,D3,60 months,Verified,87905,0.0691,589.27,0.1917,41161,7,11704
1,1000001,Telangana,INDIVIDUAL,10 years,Britannia Industries,C,MORTGAGE,4/26/2024,06/08/2024,4/20/2024,...,F2,60 months,Source Verified,35768,0.0683,214.44,0.1225,45430,3,16223
2,1000002,Maharashtra,INDIVIDUAL,< 1 year,Bharti Airtel,B,OTHER,3/23/2024,1/25/2025,01/09/2024,...,C5,60 months,Source Verified,95008,0.0612,918.99,0.1847,28540,14,12449
3,1000003,Madhya Pradesh,INDIVIDUAL,3 years,Dabur India,C,MORTGAGE,8/22/2024,04/05/2024,12/14/2024,...,D4,36 months,Verified,54660,0.1953,617.99,0.1583,6469,7,16872
4,1000004,Tamil Nadu,INDIVIDUAL,4 years,Wipro,E,RENT,03/02/2024,6/25/2024,7/31/2024,...,A1,60 months,Not Verified,103601,0.2884,142.59,0.1540,33992,20,11348


### Cleaning the special characters from columns and changing the data type of the column

In [184]:
# Renaming the column and changing the data type, removing special characters

df['emp_length_year'] = (
    df['emp_length']
    .str.replace('<', '', regex=False)
    .str.replace('years', '', regex=False)
    .str.replace('year', '', regex=False)
    .str.strip()
)

df['emp_length_year'] = df['emp_length_year'].astype('int64')

df.drop(columns=['emp_length'], inplace=True)

df.head()


,id,address_state,application_type,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,...,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length_year
0,1000000,West Bengal,INDIVIDUAL,Larsen & Toubro,E,OTHER,9/26/2024,7/29/2024,6/22/2024,Fully Paid,...,60 months,Verified,87905,0.0691,589.27,0.1917,41161,7,11704,4
1,1000001,Telangana,INDIVIDUAL,Britannia Industries,C,MORTGAGE,4/26/2024,06/08/2024,4/20/2024,Late,...,60 months,Source Verified,35768,0.0683,214.44,0.1225,45430,3,16223,10
2,1000002,Maharashtra,INDIVIDUAL,Bharti Airtel,B,OTHER,3/23/2024,1/25/2025,01/09/2024,Charged Off,...,60 months,Source Verified,95008,0.0612,918.99,0.1847,28540,14,12449,1
3,1000003,Madhya Pradesh,INDIVIDUAL,Dabur India,C,MORTGAGE,8/22/2024,04/05/2024,12/14/2024,Fully Paid,...,36 months,Verified,54660,0.1953,617.99,0.1583,6469,7,16872,3
4,1000004,Tamil Nadu,INDIVIDUAL,Wipro,E,RENT,03/02/2024,6/25/2024,7/31/2024,Late,...,60 months,Not Verified,103601,0.2884,142.59,0.1540,33992,20,11348,4


In [185]:
# change the term to int64 and removing the 'months' string

df['term'] = df['term'].str.replace('months', '', regex=False).str.strip()

df['term'] = df['term'].astype('int64')

df.head()

,id,address_state,application_type,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,...,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length_year
0,1000000,West Bengal,INDIVIDUAL,Larsen & Toubro,E,OTHER,9/26/2024,7/29/2024,6/22/2024,Fully Paid,...,60,Verified,87905,0.0691,589.27,0.1917,41161,7,11704,4
1,1000001,Telangana,INDIVIDUAL,Britannia Industries,C,MORTGAGE,4/26/2024,06/08/2024,4/20/2024,Late,...,60,Source Verified,35768,0.0683,214.44,0.1225,45430,3,16223,10
2,1000002,Maharashtra,INDIVIDUAL,Bharti Airtel,B,OTHER,3/23/2024,1/25/2025,01/09/2024,Charged Off,...,60,Source Verified,95008,0.0612,918.99,0.1847,28540,14,12449,1
3,1000003,Madhya Pradesh,INDIVIDUAL,Dabur India,C,MORTGAGE,8/22/2024,04/05/2024,12/14/2024,Fully Paid,...,36,Verified,54660,0.1953,617.99,0.1583,6469,7,16872,3
4,1000004,Tamil Nadu,INDIVIDUAL,Wipro,E,RENT,03/02/2024,6/25/2024,7/31/2024,Late,...,60,Not Verified,103601,0.2884,142.59,0.1540,33992,20,11348,4


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     40000 non-null  int64  
 1   address_state          40000 non-null  object 
 2   application_type       40000 non-null  object 
 3   emp_title              40000 non-null  object 
 4   grade                  40000 non-null  object 
 5   home_ownership         40000 non-null  object 
 6   issue_date             40000 non-null  object 
 7   last_credit_pull_date  40000 non-null  object 
 8   last_payment_date      40000 non-null  object 
 9   loan_status            40000 non-null  object 
 10  next_payment_date      40000 non-null  object 
 11  member_id              40000 non-null  int64  
 12  purpose                40000 non-null  object 
 13  sub_grade              40000 non-null  object 
 14  term                   40000 non-null  int64  
 15  ve

### Trim the rows and columns of the dataframe

In [187]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

print(df)

C:\Users\Cosmoos\AppData\Local\Temp\ipykernel_15524\3076656749.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


       id      address_state   application_type emp_title             grade  \
0      1000000     West Bengal  INDIVIDUAL            Larsen & Toubro  E      
1      1000001       Telangana  INDIVIDUAL       Britannia Industries  C      
2      1000002     Maharashtra  INDIVIDUAL              Bharti Airtel  B      
3      1000003  Madhya Pradesh  INDIVIDUAL                Dabur India  C      
4      1000004      Tamil Nadu  INDIVIDUAL                      Wipro  E      
...        ...             ...              ...                   ...   ...   
39995  1039995      Tamil Nadu  INDIVIDUAL           Bharat Petroleum  C      
39996  1039996       Karnataka  INDIVIDUAL               Asian Paints  F      
39997  1039997     West Bengal  INDIVIDUAL                 ICICI Bank  E      
39998  1039998  Madhya Pradesh  INDIVIDUAL                Tata Motors  A      
39999  1039999         Gujarat  INDIVIDUAL                        RIL  F      

      home_ownership issue_date  last_credit_pull_d

### fixing the date format from string to date

In [200]:
# verifying the data format of the date columns

date_columns = ['issue_date', 'last_credit_pull_date', 'last_payment_date', 'next_payment_date']

for column in date_columns:
    invalid_dates = df[df[column].isna()]
    print(f"Invalid or NaT values in {column}:\n", invalid_dates)


Invalid or NaT values in issue_date:
 Empty DataFrame
Columns: [id, address_state, application_type, emp_title, grade, home_ownership, issue_date, last_credit_pull_date, last_payment_date, loan_status, next_payment_date, member_id, purpose, sub_grade, term, verification_status, annual_income, dti, installment, int_rate, loan_amount, total_acc, total_payment, emp_length_year]
Index: []

[0 rows x 24 columns]
Invalid or NaT values in last_credit_pull_date:
 Empty DataFrame
Columns: [id, address_state, application_type, emp_title, grade, home_ownership, issue_date, last_credit_pull_date, last_payment_date, loan_status, next_payment_date, member_id, purpose, sub_grade, term, verification_status, annual_income, dti, installment, int_rate, loan_amount, total_acc, total_payment, emp_length_year]
Index: []

[0 rows x 24 columns]
Invalid or NaT values in last_payment_date:
 Empty DataFrame
Columns: [id, address_state, application_type, emp_title, grade, home_ownership, issue_date, last_credit_p

In [ ]:
# lets replace the null values with 0

print(df[date_columns].head())
df[column].fillna('0001-01-01', inplace=True)

  issue_date  last_credit_pull_date last_payment_date next_payment_date
0   9/26/2024   7/29/2024             6/22/2024         4/23/2024      
1   4/26/2024  06/08/2024             4/20/2024         2/19/2024      
2   3/23/2024   1/25/2025            01/09/2024        09/06/2024      
3   8/22/2024  04/05/2024            12/14/2024         1/14/2024      
4  03/02/2024   6/25/2024             7/31/2024        04/04/2024      


C:\Users\Cosmoos\AppData\Local\Temp\ipykernel_15524\3692191133.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna('0001-01-01', inplace=True)


In [ ]:
# Formated Date Columns to standartize date format (YYYY-MM-DD)

date_columns = ['issue_date', 'last_credit_pull_date', 'last_payment_date', 'next_payment_date']

for column in date_columns:

    df[column] = pd.to_datetime(df[column], errors='coerce')
    df[column] = df[column].dt.strftime('%Y-%m-%d')


In [206]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     40000 non-null  int64  
 1   address_state          40000 non-null  object 
 2   application_type       40000 non-null  object 
 3   emp_title              40000 non-null  object 
 4   grade                  40000 non-null  object 
 5   home_ownership         40000 non-null  object 
 6   issue_date             40000 non-null  object 
 7   last_credit_pull_date  40000 non-null  object 
 8   last_payment_date      40000 non-null  object 
 9   loan_status            40000 non-null  object 
 10  next_payment_date      40000 non-null  object 
 11  member_id              40000 non-null  int64  
 12  purpose                40000 non-null  object 
 13  sub_grade              40000 non-null  object 
 14  term                   40000 non-null  int64  
 15  ve

,id,address_state,application_type,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,...,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length_year
0,1000000,West Bengal,INDIVIDUAL,Larsen & Toubro,E,OTHER,2024-09-26,2024-07-29,2024-06-22,Fully Paid,...,60,Verified,87905,0.0691,589.27,0.1917,41161,7,11704,4
1,1000001,Telangana,INDIVIDUAL,Britannia Industries,C,MORTGAGE,2024-04-26,2024-06-08,2024-04-20,Late,...,60,Source Verified,35768,0.0683,214.44,0.1225,45430,3,16223,10
2,1000002,Maharashtra,INDIVIDUAL,Bharti Airtel,B,OTHER,2024-03-23,2025-01-25,2024-01-09,Charged Off,...,60,Source Verified,95008,0.0612,918.99,0.1847,28540,14,12449,1
3,1000003,Madhya Pradesh,INDIVIDUAL,Dabur India,C,MORTGAGE,2024-08-22,2024-04-05,2024-12-14,Fully Paid,...,36,Verified,54660,0.1953,617.99,0.1583,6469,7,16872,3
4,1000004,Tamil Nadu,INDIVIDUAL,Wipro,E,RENT,2024-03-02,2024-06-25,2024-07-31,Late,...,60,Not Verified,103601,0.2884,142.59,0.1540,33992,20,11348,4


### Extraction of year from the date column

In [209]:
# Extract the year from the 'issue_date' column and store it in a new column

df['issue_year'] = pd.to_datetime(df['issue_date'], errors='coerce').dt.year

print(df[['issue_date', 'issue_year']].head())


  issue_date   issue_year
0  2024-09-26  2024      
1  2024-04-26  2024      
2  2024-03-23  2024      
3  2024-08-22  2024      
4  2024-03-02  2024      


## Exporting the cleaned data to a CSV file

In [210]:
# Export the DataFrame to a CSV file
df.to_csv('business_loan_cleaned.csv', index=False)

print("DataFrame successfully exported to 'output_file.csv'")


DataFrame successfully exported to 'output_file.csv'
